# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from fastai import * # import the FastAI v3 lib which includes pytorch
from fastai.vision import  * # import all of the computer vision related libs from vision 

# lets import our necessary magic libs
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


# Constants

In [ ]:
BATCH_SIZE = 64 
IMG_SIZE = 224
WORKERS = 0 
DATA_PATH_STR = '../input/parkinsons-drawings/'
DATA_PATH_OBJ = Path(DATA_PATH_STR)

# Data Pre-Processing

In [ ]:
tfms = get_transforms() # standard data augmentation ()

data = (ImageList.from_folder(DATA_PATH_OBJ)        # get data from path object
        .split_by_rand_pct()                        # separate 20% of data for validation set
        .label_from_folder()                          # label based on directory
        .transform(tfms, size=IMG_SIZE)                   # added image data augmentation
        .databunch(bs=BATCH_SIZE, num_workers=WORKERS)    # create ImageDataBunch
        .normalize(imagenet_stats))                   # normalize RGB vals using imagenet stats


In [ ]:
('training DS size:', len(data.train_ds), 'validation DS size:' ,len(data.valid_ds))


In [ ]:
data.classes

In [ ]:
data.show_batch(rows=4, figsize=(10,8))

# Creating & Training a Resnet34

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy, model_dir='/models')

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(10, max_lr = slice(5.74e-03/10))

In [ ]:
learn.export('stage-1-93.pkl')

In [ ]:
learn.save('stage-1-93')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.top_losses()
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused()